## OOP进阶

### @property装饰器
property=属性
在类中,如果直接将属性暴漏出去是不正确的,可以通过@property包装起包装getter和setter方法
[@property的介绍与使用](https://zhuanlan.zhihu.com/p/64487092)

场景1：修饰方法,使方法可以像属性一样访问

In [17]:
class DataSet:
    def __init__(self):
        self.name = None

    @property #强调下面只是定义了一个属性 而非方法
    def with_property(self):
        return 15

    def without_property(self):
        self.name = "123"
        return self.name


l = DataSet()
print(l.with_property) # 加了property之后，直接通过调用属性的方法就可以，不需要加()
print(l.without_property())

15
123


场景2:与所定义的属性配合使用,防止属性被修改(只读)

In [19]:
class DataSet1:
    def __init__(self):
        self._images  = 1
        self._labels = 2


l = DataSet1()
print(l._labels) #可以使用_labels进行属性输出，但是此时会提示保护成员


2


In [18]:
class DataSet1:
    def __init__(self):
        self._images  = 1
        self._labels = 2
    @property
    def images(self):
        return self._images

    @property
    def labels(self):
        return self._labels

l = DataSet1()
print(l.labels) #可以使用labels进行输出，但是此时无法通过直接调用l._labels进行输出或者修改，只读

2


@property 相当于添加getter,同样可以添加setter

In [7]:
class Person:

    def __init__(self, name,age):
        self._name = name
        self._age = age

   # getter方法
    @property
    def name(self):
        return self._name

    @property
    def age(self):
        return self._age

    # setter方法
    @age.setter
    def age(self, age):
        self._age = age

    @name.setter
    def name(self, name):
        self._name = name

    def play(self):
        if self._age<16:
            print(self._name,"16")
        else:
            print(self._name,"17")


def main():
    p = Person("李",15)
    p.name="杨"

    name = p.name
    print(name)
if __name__ == "__main__":
    main()

杨


### __slots__魔法 slots插槽
Python是一门动态语言DL(Dynamic Language),通常DL允许我们在程序运行时给对象绑定或解绑新的属性或者方法.但是如果我们需要限定自定义类型的对象只能绑定某些属性,通过__slots__变量进行限定，需要注意的是限定只针对当前对象生效,而不是类或子类

In [22]:
class Person(object):
    __slots__ = ('_name', '_age', '_gender')
    # 限定Person的对象只能绑定这三个属性

    def __init__(self, name, age):
        self._name = name
        self._age = age

    @property
    def name(self):
        return self._name

    @property
    def age(self):
        return self._age

    @age.setter
    def age(self, age):
        self._age = age

    def play(self):
        if self._age <= 16:
            print('%s正在玩飞行棋.' % self._name)
        else:
            print('%s正在玩斗地主.' % self._name)
def main():
    person = Person("李梦洋",22)
    print(person.name)
    person._gender = '男' # 动态添加
    print(person._gender)  #这里可以正常输出，因为_gender属性是在slots范围内的，可以动态添加
    person._is_girl = True # 这里会报错，_is_girl 不在slots范围内。  'Person' object has no attribute '_is_girl'

if __name__ == '__main__':
    main()

李梦洋
男


AttributeError: 'Person' object has no attribute '_is_girl'

## 静态方法和类方法
在写类的方法并不需要都是对象方法,我们定义一个“三角形”类，通过传入三条边长来构造三角形，并提供计算周长和面积的方法，但是传入的三条边长未必能构造出三角形对象，因此我们可以**先写一个方法来验证三条边长是否可以构成三角形**,这个方法很显然就不是对象方法,可以使用静态方法来解决这个问题.

In [3]:
from math import sqrt

class Triangle:
    def __init__(self,a,b,c):
        self._a = a
        self._b = b
        self._c = c

    @property
    def a(self):
        return self._a
    @property
    def b(self):
        return self._b
    @property
    def c(self):
        return self._c

    @staticmethod
    def is_valid(a,b,c):
        """
        静态方法 判断是否可以组成三角形
        :param a:
        :param b:
        :param c:
        :return:
        """
        return a+b>c and a+c>b and b+c>a

    def perimeter(self):
        return self.a+self.b+self.c
def main():
    a ,b ,c =3,4,5
    if Triangle.is_valid(a,b,c):
        t = Triangle(a, b, c)
        print(t.perimeter())

    else:
        print("Triangle is invalid")
        
def main1():
    a ,b ,c =3,4,3
    t = Triangle(a, b, c)
    print(t.perimeter())
    # 这样仍可以运行 ，但是不严谨，最好是需要通过静态方法提前判断

if __name__ == '__main__':
    main1()

10


Python可以在类中定义类方法,类方法的第一个参数约定名为cls，它代表的是当前类相关的信息的对象（类本身也是一个对象，有的地方也称之为类的元数据对象），通过这个参数我们可以获取和类相关的信息并且可以创建出类的对象,代码如下所示. @classmethod方法不需要实力化也可以调用

In [4]:
from time import time,localtime,sleep

class Clock(object):
    def __init__(self,hour, minute,second):
        self._hour = hour
        self._minute = minute
        self._second = second

    @classmethod
    def now(cls):
        """
        类方法，非对象才能执行
        将当前的时间作为参数传递给类
        :return: 返回一个Clock对象
        """
        ctime = localtime(time())
        return  cls(ctime.tm_hour,ctime.tm_min,ctime.tm_sec)

    def run(self):
        """

        :return:
        """
        self._second += 1
        if self._second == 60:
            self._second = 0
            self._minute += 1
            if self._minute == 60:
                self._minute = 0
                self._hour += 1
                if self._hour == 24:
                    self._hour = 0

    def show(self):
        """显示时间"""
        return '%02d:%02d:%02d' % \
               (self._hour, self._minute, self._second)

def main():
    print(f"当前的时间{Clock.now().show()}")
    # time = Clock.now()

    # while True:
    #     print(time.show())
    #     sleep(1)
    #     time.run()

if __name__ == '__main__':
    main()

当前的时间00:53:07


### 类与类的关系

* 继承和泛化 学生与人 手机和电子产品
* $关联=\left\{\begin{matrix}聚合关系\\合成关系\end{matrix}\right.$
部门和员工 汽车和引擎 如果是整体和部分的关系 称为**聚合关系** 如果整体和部分不可分割 称为**合成关系**
* 依赖 司机的驾驶行为中调用了汽车这个类,则称司机和汽车为依赖关系

利用类间关系可以实现代码的复用,减少代码量

### 继承和多态
* 提供继承信息的称为父类,基类,超类
* 得到继承信息的称为子类,派生类,衍生类
* 子类可以得到父类的属性与方法,也可以自定义新的属性和方法
[里氏替换原则](https://zh.wikipedia.org/wiki/里氏替换原则):子类对象去替换掉一个父类对象

In [28]:
class Person(object):
    """
    人
    """

    def __init__(self, name, age):
        self._name = name
        self._age = age
        self._identity = "Person"



    @property
    def name(self):
        return self._name

    @property
    def age(self):
        return self._age

    @age.setter
    def age(self, age):
        self._age = age

    @name.setter
    def name(self, name):
        self._name = name

    def play(self):
        print(f"{self.name}正在愉快地玩耍。")

    def watch_movie(self):
        if self.age >= 18:
            print(f"{self.name}正在看片。")
        else:
            print(f"{self.name}正在看动画片。")

In [17]:
class Student(Person):
    def __init__(self, name, age, grade):
        super().__init__(name, age)
        self._grade = grade
        self._identity = "Student"

    @property
    def grade(self):
        return self._grade

    @grade.setter
    def grade(self, grade):
        self._grade = grade

    def study(self, *course_name):
        print(f"{self.grade}的{self.name}正在学习", end="")
        for course in course_name:
            print(f"{course}",end="")
        print()

In [15]:
class Teacher(Person):
    def __init__(self, name, age, title):
        super().__init__(name, age)
        self._title = title
        self._identity = "Teacher"

    @property
    def title(self):
        return self._title

    @title.setter
    def title(self, title):
        self._title = title

    def teach(self, course):
        print('%s%s正在讲%s.' % (self._name, self._title, course))

In [21]:
def main():
    stu1 = Student("王大锤",15,"五年级")
    stu1.study("数学","语文")
    stu1.watch_movie() # 这个方法是父类的方法
    t = Teacher("李梦洋",22,"废物")
    t.teach("Python设计")
    t.watch_movie()

if __name__ == '__main__':
    main()

五年级的王大锤正在学习数学语文
王大锤正在看动画片。
李梦洋废物正在讲Python设计.
李梦洋正在看片。
男


In [27]:
class Student1(Student):

    def __init__(self, name, age, grade):
        super().__init__(name, age, grade)
        self._grade = grade

    @staticmethod
    def is_valid(new_age):
        """
        静态方法，判断年龄是否合适
        :param new_age: 学生的年龄
        :return: 是否符合条件
        """
        return 18 > new_age > 3

    def watch_movie(self):
        """
        对父类方法的重写
        :return:直接输出不能看电视
        """
        print(f"{self._grade}的{self.name}同学只是一名学生，不能看电影哦")

In [26]:
name = "小赵"
age = 15
grade = "三年级"
if Student1.is_valid(age):
    stu2 = Student1(name,age,grade)
    stu2.watch_movie()
else:
    print("孩子年龄不符合标准不能上学哦")

三年级的小赵同学只是一名学生，不能看电影哦


#### 抽象类与抽象方法@abstractmethod
[abc模块](https://docs.python.org/zh-cn/3/library/abc.html)

In [29]:
from abc import ABCMeta, abstractmethod
class Pet(object,metaclass=ABCMeta):
    """宠物"""

    def __init__(self, nickname):
        self._nickname = nickname

    @abstractmethod
    def make_voice(self):
        """发出声音"""
        """抽象方法，所有继承Pet的子类都必须重新定义该方法"""
        pass

class Dog(Pet):
    def make_voice(self):
        print("汪汪汪")

class Cat(Pet):
    def make_voice(self):
        print("喵喵喵")
def main():
    dog = Dog("旺")

    cat = Cat("花")
    dog.make_voice()
    cat.make_voice()
if __name__ == '__main__':
    main()


汪汪汪
喵喵喵


### 多继承
在父类1和父类2中 如果出现方法相同,则会根据[MRO规则](https://blog.csdn.net/qq_38923792/article/details/94414944)进行查找,应尽量减少这种代码的产生
![MRO](./res/mro.png)

In [30]:
class A(object):
    @staticmethod
    def test_1():
        print("A.test_1()")

    @staticmethod
    def test_2():
        print("A.test_2()")

class B(object):
    @staticmethod
    def test_1():
        print("B.test_1()")

class C(A,B):
    # 多继承
    pass
def main():
    c= C()
    c.test_1() # A.test_1()
    c.test_2()

if __name__ == '__main__':
    main()

A.test_1()
A.test_2()


In [32]:
class A1: pass
class A2: pass
class A3: pass
class B1(A1,A2): pass
class B2(A2): pass
class B3(A2,A3): pass
class C1(B1): pass
class C2(B1,B2): pass
class C3(B2,B3): pass
class D(C1, C2, C3): pass

print("从D开始查找：")
for s in D.__mro__:
    print(s)
    """
    <class '__main__.D'>
    <class '__main__.C1'>
    <class '__main__.C2'>
    <class '__main__.B1'>
    <class '__main__.A1'>
    <class '__main__.C3'>
    <class '__main__.B2'>
    <class '__main__.B3'>
    <class '__main__.A2'>
    <class '__main__.A3'>
    <class 'object'>
    """

从D开始查找：
<class '__main__.D'>
<class '__main__.C1'>
<class '__main__.C2'>
<class '__main__.B1'>
<class '__main__.A1'>
<class '__main__.C3'>
<class '__main__.B2'>
<class '__main__.B3'>
<class '__main__.A2'>
<class '__main__.A3'>
<class 'object'>


### 综合案例：[奥特曼打怪兽](./奥特曼打怪兽.py)